In [90]:
import numpy as np
from numpy import asarray as npa
import keras
import tensorflow as tf
import re
from string import ascii_lowercase
tf.logging.set_verbosity(tf.logging.ERROR)

In [59]:
ds = open('sherlock.txt')

In [60]:
d = ds.read()
d = d[:10000]
data = re.sub('[^A-Za-z]',' ', d)

In [88]:
data = data.lower()
print len(data)

10000


In [107]:
#Find unique values in x
#Create a dixtionary and map chars of x to an index
dct = list(set(data))
#create a reverse dictionary and map indices back to chars
revdct = [(j,i) for i,j in enumerate(dct)]
revdct = dict(revdct)

In [108]:
print revdct

{' ': 0, 'a': 1, 'c': 2, 'b': 3, 'e': 4, 'd': 5, 'g': 6, 'f': 7, 'i': 8, 'h': 9, 'k': 10, 'j': 11, 'm': 12, 'l': 13, 'o': 14, 'n': 15, 'q': 16, 'p': 17, 's': 18, 'r': 19, 'u': 20, 't': 21, 'w': 22, 'v': 23, 'y': 24, 'x': 25, 'z': 26}


In [150]:
inputSize = len(data)
blockSize = 100
max_features = 27
rowSize = inputSize/blockSize
xt = np.zeros((rowSize, blockSize, 27))

In [151]:
for i in range(rowSize):
    k = i*25
    for j in range(blockSize):
        xt[i,j,revdct[data[k+j]]] = 1

In [152]:
yt = np.copy(xt)

x = np.roll(yt, 1, axis=1)
x[:, 0, :] = 0
x[:, 0, 0] = 1

In [155]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(blockSize,  max_features)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(max_features))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_10 (LSTM)                   (None, 100, 256)      290816      lstm_input_4[0][0]               
____________________________________________________________________________________________________
lstm_11 (LSTM)                   (None, 100, 256)      525312      lstm_10[0][0]                    
____________________________________________________________________________________________________
lstm_12 (LSTM)                   (None, 100, 256)      525312      lstm_11[0][0]                    
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 100, 27)       6939        lstm_12[0][0]                    
___________________________________________________________________________________________

In [156]:
hist = model.fit(x, yt, batch_size=64, nb_epoch=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 3s - loss: 3.0152     
Epoch 2/10
100/100 [==============================] - 2s - loss: 2.9703     
Epoch 3/10
100/100 [==============================] - 3s - loss: 2.3586     
Epoch 4/10
100/100 [==============================] - 3s - loss: 2.2945     
Epoch 5/10
100/100 [==============================] - 3s - loss: 2.2393     
Epoch 6/10
100/100 [==============================] - 3s - loss: 2.2405     
Epoch 7/10
100/100 [==============================] - 3s - loss: 2.2259     
Epoch 

In [163]:
import numpy
def mnrnd(probs):
    rnd = numpy.random.random()
    for i in xrange(len(probs)):
        rnd -= probs[i]
        if rnd <= 0:
            return i
    return i

sentences = numpy.zeros((128, blockSize, max_features))
sentences[:, 0, 0] = 1

# Start sampling char-sequences. At each iteration i the probability over
# the i-th character of each sequences is computed. 
for i in numpy.arange(10):
    probs = model.predict_proba(sentences, verbose=2)[:,i,:]
    # Go over each sequence and sample the i-th character.
    for j in numpy.arange(len(sentences)):
        sentences[j, i+1, mnrnd(probs[j, :])] = 1
sentences = [sentence[1:].nonzero()[1] for sentence in sentences]

# Convert to readable text.
text = []
for sentence in sentences:
    text.append(''.join([dct[word] for word in sentence]))
    
print text

['lledzt  e ', 'giy g  gtm', 'gskk  k hl', 'xd c et  n', 'nlgafri vc', 'oqfrazdca ', 'dyxkzy d n', 'zjayb odt ', 'xprtvvqrtl', 'tmxtu uo  ', 'lmqga     ', 'ajvazrdcea', 'dukiri p o', 'fywncu   o', 'tg cijtn  ', 'ankqeegnn ', 'xhseuuohi ', 'qrycyvyiv ', 's xlvgvp p', ' pllznt s ', 'luedyf gte', 'inkwdn a  ', 'oqjqgue   ', 'pu jl  ie ', 'aezi tdoe ', 'cfabub s  ', 'gqd i e tt', 'p vvnj aau', ' g irt  v ', 'esjrnutdce', 'zgzvukxe e', 'wpve slmnk', 'kzkfyy  a ', 'cazvdh ot ', 'wm edh led', 'cbkuitn  e', 'vqnoccta  ', 'jutvfssa n', 'yonoa dirs', 'nnruitdo n', 'hbc n  ee ', 'qgwh e tur', 'kbgct  nee', 's ddve vu ', 'gxkfiru cu', 'lv tyizoen', 'azqc hn   ', 'hirh  zts ', 'vnnr e mhn', 'uwjce  ay ', 'mxeis  eet', 'prvk ensfh', 'wcuoa  ct ', ' yoc  c h ', 'ebiwuth bi', 'wolxgcsoi ', 'dkmk tore ', 'msy fmas  ', 'talo  a  s', 'luvo gia h', 'xhrx a d l', 'f yzltut  ', 'pbcddd  pi', 'gksvcoano ', 'cjsv evrbb', 'jdwvw ysto', 'ljafata tk', 'oto eqkets', 'auhgmchgi ', 'cdkalbz xe', 'ivuveri  c', 'lira